In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv("/content/drive/MyDrive/nyc_311_clean.csv")

/tmp/ipython-input-798666728.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/nyc_311_clean.csv")


# 1. Text Representation (Embeddings)

In [5]:
df["text"] = (
    df["complaint_type"].fillna("") + " | " +
    df["descriptor"].fillna("") + " | " +
    df["descriptor_2"].fillna(""))

In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(
    "all-MiniLM-L6-v2",
    device="cuda"
)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
model = model.half()
embeddings = model.encode(
    df["text"].values,
    batch_size=1024,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

Batches:   0%|          | 0/3760 [00:00<?, ?it/s]

In [8]:
embeddings.shape

(3850000, 384)

In [9]:
import numpy as np

emb = embeddings.astype(np.float32)
emb /= (np.linalg.norm(emb, axis=1, keepdims=True) + 1e-12)

In [10]:
rng = np.random.default_rng(42)
CLUSTER_N = 300_000  # 200k–500k is a good range
idx = rng.choice(len(emb), size=CLUSTER_N, replace=False)

X = emb[idx]
df_X = df.iloc[idx].copy()

In [12]:
from sklearn.cluster import MiniBatchKMeans

K = 200  # start 100–400 for 311-scale corpora
kmeans = MiniBatchKMeans(
    n_clusters=K,
    batch_size=8192,
    random_state=42,
    n_init="auto",
    max_no_improvement=20,
    verbose=0
)
sample_cluster = kmeans.fit_predict(X)
df_X["cluster"] = sample_cluster

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

vectorizer = TfidfVectorizer(
    max_features=80_000,
    ngram_range=(1,2),
    stop_words="english",
    min_df=5
)
tfidf = vectorizer.fit_transform(df_X["text"].astype(str))

terms = np.array(vectorizer.get_feature_names_out())

cluster_top_terms = {}
for c in range(K):
    rows = np.where(df_X["cluster"].values == c)[0]
    if len(rows) == 0:
        continue
    # average tf-idf within cluster
    mean_tfidf = tfidf[rows].mean(axis=0).A1
    top = mean_tfidf.argsort()[-12:][::-1]
    cluster_top_terms[c] = terms[top].tolist()

cluster_summary = (
    pd.DataFrame({
        "cluster": list(cluster_top_terms.keys()),
        "size_in_sample": [int((df_X["cluster"]==c).sum()) for c in cluster_top_terms.keys()],
        "top_terms": [", ".join(cluster_top_terms[c][:10]) for c in cluster_top_terms.keys()]
    })
    .sort_values("size_in_sample", ascending=False)
)
cluster_summary.head(20)


,cluster,size_in_sample,top_terms
185,192,23642,"residential loud, noise residential, music par..."
171,178,13882,"blocked hydrant, hydrant, parking blocked, blo..."
8,9,11425,"pounding, banging, banging pounding, residenti..."
93,97,11112,"parking, parking sign, posted parking, parking..."
142,148,11026,"heat, building heat, water entire, entire buil..."
4,4,10628,"driveway access, blocked driveway, access, dri..."
19,20,10622,"sidewalk loud, noise street, street sidewalk, ..."
3,3,7378,"hot water, hot, water, heat hot, heat, water e..."
24,25,7056,"blocked sidewalk, parking blocked, sidewalk, b..."
46,48,6352,"heat, apartment heat, water apartment, apartme..."


In [14]:
# cosine similarity to centroids
centroids = kmeans.cluster_centers_.astype(np.float32)
centroids /= (np.linalg.norm(centroids, axis=1, keepdims=True) + 1e-12)

# scores: (N x K)
scores = emb @ centroids.T
all_cluster = scores.argmax(axis=1)

df["cluster"] = all_cluster

In [34]:
len(df["cluster"].unique())

192

In [35]:
THEMES = [
    "Residential Noise Disturbances",
    "Street & Public Space Noise",
    "Commercial Noise Violations",
    "Illegal Parking & Traffic Obstructions",
    "Heating & Hot Water Failures",
    "Sanitation & Waste Conditions",
    "Vehicle Abandonment & Derelict Vehicles",
    "Street & Traffic Infrastructure Failures",
    "Public Health & Sanitary Conditions",
    "Public Safety & Social Services",
    "Parks & Public Facilities Maintenance",
]


In [36]:
MANUAL_THEME_MAP = {
    # Residential Noise
    192: "Residential Noise Disturbances",
    9:   "Residential Noise Disturbances",
    102: "Residential Noise Disturbances",

    # Street / Public Noise
    20:  "Street & Public Space Noise",
    40:  "Street & Public Space Noise",
    55:  "Street & Public Space Noise",
    38:  "Street & Public Space Noise",

    # Commercial Noise
    122: "Commercial Noise Violations",

    # Illegal Parking
    178: "Illegal Parking & Traffic Obstructions",
    97:  "Illegal Parking & Traffic Obstructions",
    4:   "Illegal Parking & Traffic Obstructions",
    13:  "Illegal Parking & Traffic Obstructions",
    25:  "Illegal Parking & Traffic Obstructions",
    190: "Illegal Parking & Traffic Obstructions",
    12:  "Illegal Parking & Traffic Obstructions",
    177: "Illegal Parking & Traffic Obstructions",

    # Heat / Hot Water
    148: "Heating & Hot Water Failures",
    3:   "Heating & Hot Water Failures",
    48:  "Heating & Hot Water Failures",
    171: "Heating & Hot Water Failures",

    # Sanitation
    23:  "Sanitation & Waste Conditions",
    10:  "Sanitation & Waste Conditions",
    8:   "Sanitation & Waste Conditions",

    # Abandoned Vehicles
    119: "Vehicle Abandonment & Derelict Vehicles",
    14:  "Vehicle Abandonment & Derelict Vehicles",

    # Infrastructure
    184: "Street & Traffic Infrastructure Failures",
    36:  "Street & Traffic Infrastructure Failures",
    35:  "Street & Traffic Infrastructure Failures",

    # Public Health
    7:   "Public Health & Sanitary Conditions",
    139: "Public Health & Sanitary Conditions",
    18:  "Public Health & Sanitary Conditions",

    # Public Safety / Social
    83:  "Public Safety & Social Services",
    16:  "Public Safety & Social Services",
    15:  "Public Safety & Social Services",

    # Parks / Facilities
    113: "Parks & Public Facilities Maintenance",
}


In [37]:
THEME_KEYWORDS = {
    "Residential Noise Disturbances": [
        "residential", "loud", "music", "party", "banging", "pounding"
    ],
    "Street & Public Space Noise": [
        "street", "sidewalk", "vehicle", "truck", "car", "construction"
    ],
    "Commercial Noise Violations": [
        "commercial", "restaurant", "bar", "club"
    ],
    "Illegal Parking & Traffic Obstructions": [
        "parking", "blocked", "hydrant", "driveway", "crosswalk", "double"
    ],
    "Heating & Hot Water Failures": [
        "heat", "hot water", "boiler", "no heat"
    ],
    "Sanitation & Waste Conditions": [
        "trash", "garbage", "litter", "dumping", "dirty"
    ],
    "Vehicle Abandonment & Derelict Vehicles": [
        "abandoned", "derelict", "vehicle"
    ],
    "Street & Traffic Infrastructure Failures": [
        "pothole", "street light", "signal", "traffic"
    ],
    "Public Health & Sanitary Conditions": [
        "rodent", "rat", "mice", "mold", "pests", "unsanitary"
    ],
    "Public Safety & Social Services": [
        "drug", "encampment", "homeless", "assistance"
    ],
    "Parks & Public Facilities Maintenance": [
        "park", "facility", "playground", "maintenance"
    ],
}


In [38]:
def assign_theme(cluster_id, top_terms):
    # 1) Manual override
    if cluster_id in MANUAL_THEME_MAP:
        return MANUAL_THEME_MAP[cluster_id]

    # 2) Keyword-based scoring
    term_string = str(top_terms).lower()
    scores = {}

    for theme, keywords in THEME_KEYWORDS.items():
        scores[theme] = sum(kw in term_string for kw in keywords)

    # 3) Pick best match
    best_theme = max(scores, key=scores.get)

    # 4) Fallback safety (should rarely trigger)
    if scores[best_theme] == 0:
        return "Public Safety & Social Services"

    return best_theme


In [39]:
cluster_summary["theme"] = cluster_summary.apply(
    lambda r: assign_theme(r["cluster"], r["top_terms"]),
    axis=1
)


In [40]:
# Ensure all clusters are mapped
cluster_summary["theme"].isna().sum()


np.int64(0)

In [41]:
# Check coverage
cluster_summary.groupby("theme")["cluster"].nunique().sort_values(ascending=False)


,cluster
theme,
Public Safety & Social Services,73
Street & Public Space Noise,39
Illegal Parking & Traffic Obstructions,21
Residential Noise Disturbances,14
Sanitation & Waste Conditions,13
Public Health & Sanitary Conditions,10
Heating & Hot Water Failures,7
Street & Traffic Infrastructure Failures,7
Commercial Noise Violations,3


In [43]:
df.head()

,unique_key,created_date,status,agency,agency_name,complaint_type,descriptor,borough,city,incident_zip,...,longitude,community_board,council_district,police_precinct,descriptor_2,resolution_description,closed_date,text,cluster,month
0,67513805,2026-01-17 02:05:59,In Progress,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,BROOKLYN,BROOKLYN,11209.0,...,-74.023027,10 BROOKLYN,47.0,Precinct 68,NaN,NaN,NaN,Illegal Parking | Blocked Hydrant |,6,2026-01
1,67512397,2026-01-17 02:05:27,In Progress,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,BRONX,BRONX,10458.0,...,-73.892495,07 BRONX,15.0,Precinct 52,NaN,NaN,NaN,Noise - Commercial | Loud Music/Party |,122,2026-01
2,67509505,2026-01-17 02:05:26,In Progress,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Talking,MANHATTAN,NEW YORK,10002.0,...,-73.992020,03 MANHATTAN,1.0,Precinct 5,NaN,NaN,NaN,Noise - Street/Sidewalk | Loud Talking |,40,2026-01
3,67515109,2026-01-17 02:05:13,In Progress,NYPD,New York City Police Department,Blocked Driveway,No Access,QUEENS,CORONA,11368.0,...,-73.858800,03 QUEENS,21.0,Precinct 115,NaN,NaN,NaN,Blocked Driveway | No Access |,4,2026-01
4,67516547,2026-01-17 02:05:11,In Progress,NYPD,New York City Police Department,Noise - Commercial,Loud Talking,MANHATTAN,NEW YORK,10003.0,...,-73.986488,03 MANHATTAN,2.0,Precinct 9,NaN,NaN,NaN,Noise - Commercial | Loud Talking |,1,2026-01


In [45]:
cluster_theme_lookup = cluster_summary[["cluster", "theme"]].drop_duplicates()
cluster_theme_lookup.shape

(193, 2)

In [50]:
df["theme"] = df["cluster"].map(
    dict(zip(cluster_theme_lookup["cluster"], cluster_theme_lookup["theme"])))
df["theme"].isna().sum()

np.int64(178909)

In [51]:
# Drop rows where theme is missing
df = df.dropna(subset=["theme"]).reset_index(drop=True)

In [52]:
df.columns

Index(['unique_key', 'created_date', 'status', 'agency', 'agency_name',
       'complaint_type', 'descriptor', 'borough', 'city', 'incident_zip',
       'street_name', 'incident_address', 'latitude', 'longitude',
       'community_board', 'council_district', 'police_precinct',
       'descriptor_2', 'resolution_description', 'closed_date', 'text',
       'cluster', 'month', 'theme'],
      dtype='object')

In [63]:
df["created_date"] = pd.to_datetime(df["created_date"])
df["closed_date"] = pd.to_datetime(df["closed_date"])

df["hour"] = df["created_date"].dt.hour
df["day_of_week"] = df["created_date"].dt.dayofweek
df["is_weekend"] = df["day_of_week"].isin([5,6]).astype(int)
df["day"] = df["created_date"].dt.day


In [64]:
df["resolution_hours"] = (
    df["closed_date"] - df["created_date"]
).dt.total_seconds() / 3600

df["is_open"] = df["status"].ne("Closed").astype(int)
df["slow_resolution"] = (df["resolution_hours"] > df["resolution_hours"].median()).astype(int)


In [109]:
slow_res = df[["status","slow_resolution","theme","borough","month"]]

In [67]:
zip_month = (
    df.groupby(["incident_zip", "month"])
      .size()
      .reset_index(name="count")
      .sort_values(["incident_zip", "month"])
)

zip_month["zip_rolling_mean"] = (
    zip_month
    .groupby("incident_zip")["count"]
    .transform(lambda x: x.rolling(3, min_periods=1).mean())
)

zip_month["zip_spike"] = (
    zip_month["count"] > 1.75 * zip_month["zip_rolling_mean"]
).astype(int)
df = df.merge(
    zip_month[["incident_zip", "month", "zip_spike"]],
    on=["incident_zip", "month"],
    how="left"
)


In [71]:
month_theme_df = (
    df.pivot_table(
        index="month",
        columns="theme",
        values="unique_key",   # any non-null column works
        aggfunc="count",
        fill_value=0
    )
    .sort_index()
    .reset_index()
)


In [72]:
month_theme_df.head()

theme,month,Commercial Noise Violations,Heating & Hot Water Failures,Illegal Parking & Traffic Obstructions,Parks & Public Facilities Maintenance,Public Health & Sanitary Conditions,Public Safety & Social Services,Residential Noise Disturbances,Sanitation & Waste Conditions,Street & Public Space Noise,Street & Traffic Infrastructure Failures,Vehicle Abandonment & Derelict Vehicles
0,2024-12,2121,28466,20217,405,4908,26689,28225,6385,12365,4485,3392
1,2025-01,4723,67734,47779,1083,12513,61169,78092,13266,30116,10505,9394
2,2025-02,4344,41075,42146,1098,10090,51857,32799,11809,25959,11770,8284
3,2025-03,5623,26593,52011,1744,11784,59512,36327,14850,33863,11277,10292
4,2025-04,5952,17085,48062,2134,11881,61764,37496,14972,39604,9871,9893


In [73]:
month_theme_df.to_csv("theme_counts.csv", index=False)

In [74]:
from google.colab import files
files.download("theme_counts.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [68]:
df.head(10)

,unique_key,created_date,status,agency,agency_name,complaint_type,descriptor,borough,city,incident_zip,...,month,theme,hour,day_of_week,is_weekend,day,resolution_hours,is_open,slow_resolution,zip_spike
0,67512397,2026-01-17 02:05:27,In Progress,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,BRONX,BRONX,10458.0,...,2026-01,Commercial Noise Violations,2,5,1,17,NaN,1,0,0.0
1,67509505,2026-01-17 02:05:26,In Progress,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Talking,MANHATTAN,NEW YORK,10002.0,...,2026-01,Street & Public Space Noise,2,5,1,17,NaN,1,0,0.0
2,67515109,2026-01-17 02:05:13,In Progress,NYPD,New York City Police Department,Blocked Driveway,No Access,QUEENS,CORONA,11368.0,...,2026-01,Illegal Parking & Traffic Obstructions,2,5,1,17,NaN,1,0,0.0
3,67516547,2026-01-17 02:05:11,In Progress,NYPD,New York City Police Department,Noise - Commercial,Loud Talking,MANHATTAN,NEW YORK,10003.0,...,2026-01,Residential Noise Disturbances,2,5,1,17,NaN,1,0,0.0
4,67508045,2026-01-17 02:03:13,In Progress,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,BROOKLYN,BROOKLYN,11214.0,...,2026-01,Residential Noise Disturbances,2,5,1,17,NaN,1,0,0.0
5,67508051,2026-01-17 02:02:44,In Progress,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,MANHATTAN,NEW YORK,10027.0,...,2026-01,Residential Noise Disturbances,2,5,1,17,NaN,1,0,0.0
6,67509479,2026-01-17 02:02:15,In Progress,NYPD,New York City Police Department,Illegal Parking,Posted Parking Sign Violation,BRONX,BRONX,10463.0,...,2026-01,Illegal Parking & Traffic Obstructions,2,5,1,17,NaN,1,0,0.0
7,67517916,2026-01-17 02:01:28,In Progress,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,QUEENS,CAMBRIA HEIGHTS,11411.0,...,2026-01,Residential Noise Disturbances,2,5,1,17,NaN,1,0,0.0
8,67512388,2026-01-17 02:01:22,In Progress,NYPD,New York City Police Department,Illegal Parking,Double Parked Blocking Traffic,BROOKLYN,BROOKLYN,11226.0,...,2026-01,Illegal Parking & Traffic Obstructions,2,5,1,17,NaN,1,0,0.0
9,67516482,2026-01-17 02:00:54,In Progress,NYPD,New York City Police Department,Blocked Driveway,Partial Access,BROOKLYN,BROOKLYN,11236.0,...,2026-01,Illegal Parking & Traffic Obstructions,2,5,1,17,NaN,1,0,0.0


In [66]:
theme_month.head()

,theme,month,count,rolling_mean,emergence_ratio,emerging_theme_flag
0,Commercial Noise Violations,2024-12,2121,2121.000000,1.000000,0
1,Commercial Noise Violations,2025-01,4723,3422.000000,1.380187,0
2,Commercial Noise Violations,2025-02,4344,3729.333333,1.164819,0
3,Commercial Noise Violations,2025-03,5623,4896.666667,1.148332,0
4,Commercial Noise Violations,2025-04,5952,5306.333333,1.121678,0


In [105]:
borough_coords = pd.DataFrame({
    "borough": ["MANHATTAN", "BROOKLYN", "QUEENS", "BRONX", "STATEN ISLAND"],
    "lat1": [40.7831, 40.6782, 40.7282, 40.8448, 40.5795],
    "lon1": [-73.9712, -73.9442, -73.7949, -73.8648, -74.1502]
})

df = df.merge(borough_coords, on="borough", how="left")


In [106]:
borough_coords = df[["status","lat1", "lon1","theme","borough","month","city"]]

In [110]:
slow_res.to_csv("slow_res.csv", index=False)

In [61]:
import os
os.path.exists("borough_coords.csv")


True

In [111]:
from google.colab import files
files.download("slow_res.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>